In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import nltk
import json
import csv
from tqdm import tqdm
from nltk.tokenize import word_tokenize
import string
import re
import time
import pandas as pd

# nltk.download('punkt')

#### **Tagging dataset entries using dictionary**

In [2]:
#reading a tab seperated files 
word_dictionary = pd.read_csv("dictionary.tsv",delimiter = "\t")

In [3]:
#giving column names to each column present in our table
word_dictionary.columns = ["word_type","num","word","pos","y/n","p/n"]
word_dictionary.head()

,word_type,num,word,pos,y/n,p/n
0,weaksubj,1,abandonment,noun,n,negative
1,weaksubj,1,abandon,verb,y,negative
2,strongsubj,1,abase,verb,y,negative
3,strongsubj,1,abasement,anypos,y,negative
4,strongsubj,1,abash,verb,y,negative


In [4]:
#form a pair of word and p/n inside word_state_pair list
word_state_pair = []
for i in range(len(word_dictionary)):
    temp = []
    temp.append(word_dictionary.loc[i,"word"])
    temp.append(word_dictionary.loc[i,"p/n"])
    word_state_pair.append(temp)
print(word_state_pair[:3])

[['abandonment', 'negative'], ['abandon', 'negative'], ['abase', 'negative']]


In [5]:
#checking if there are any null strings present in tweet_text columns and if present then remove them
dep = pd.read_csv("1-output-depressive.csv")
dep['tweet_text'].dropna(inplace=True)
dep = dep.dropna(how='any') 
print(np.where(pd.isnull(dep)))
print(np.where(dep.applymap(lambda x: x == '')))
print(len(dep))
dep.to_csv("1-output-depressive.csv",index=True)

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
7465


In [6]:
#tagging of tweets present output output-depressive.csv file
dep = pd.read_csv("1-output-depressive.csv")

k = []
data_dict_depressive ={}
some_milby = []

print("length of depressive tweets dataset {}".format(len(dep)))
for i in range(len(dep)):
    k.append(dep.loc[i,"id"])

def add_polarity_depressive_table():  
    counter = 0
    ids = []
    texts =[]
    pol = []
    time_stamp = []
    for i in tqdm(range(len(dep))):
            tweet_token = dep.loc[i,"tweet_text"]
            time = dep.loc[i,"timestamp"]
            
            token = word_tokenize(tweet_token)
            sumnum = 0
            sum_word = 0
            for t in token:
                for d in word_state_pair:
                    if t == d[0]:
                        sentiment = d[1]
                        if sentiment == "positive":
                            sumnum += 1
                            sum_word += 1
                        elif sentiment == "negative":
                            sumnum += -1
                            sum_word += 1
                        else:
                            sumnum += 0
                            sum_word += 1
                        break
            if sum_word != 0.0:
                sum_more = sumnum / sum_word
                if sum_more >= 0.3:
                    sum_more = 1   
                else:
                    sum_more = 0
                    
            varid = k[counter]
            ids.append(varid)
            time_stamp.append(time)
            texts.append(tweet_token)
            pol.append(sum_more)
            counter += 1
            
    data_dict_depressive['ids'] = ids
    data_dict_depressive['time'] = time_stamp
    data_dict_depressive['tweet_text'] = texts
    data_dict_depressive['tag'] = pol


length of depressive tweets dataset 7465


In [7]:
add_polarity_depressive_table()

100%|██████████| 7465/7465 [01:02<00:00, 119.55it/s]


In [8]:
print(len(data_dict_depressive['ids']) , len(data_dict_depressive['time']) , len(data_dict_depressive['tweet_text']) , len(data_dict_depressive['tag']))

7465 7465 7465 7465


In [9]:
NonDep = pd.read_csv("1-output-normal.csv")
NonDep['tweet_text'].dropna(inplace=True)
NonDep = NonDep.dropna(how='any') 
print( np.where(pd.isnull(NonDep)) )
print(np.where(NonDep.applymap(lambda x: x == '')))
print(len(NonDep))
NonDep.to_csv("1-output-normal.csv",index=True)

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
9056


In [10]:
NonDep = pd.read_csv("1-output-normal.csv")
NonDep['tweet_text'].dropna(inplace=True)
NonDep['tweet_text'].isna().sum()

0

In [11]:
#tagging of tweets present output output-depressive.csv file
#  = pd.read_csv("1-output-normal.csv")

k = []
data_dict_normal ={}
some_milby = []

print("length of normal tweets dataset {}".format(len(NonDep)))
for i in range(len(NonDep)):
    k.append(NonDep.loc[i,"id"])

def add_polarity_normal_table():  
    counter = 0
    ids = []
    texts =[]
    pol = []
    time_stamp = []
    for i in tqdm(range(len(NonDep))):
            tweet_token = NonDep.loc[i,"tweet_text"]
            time = NonDep.loc[i,"timestamp"]
            token = word_tokenize(tweet_token)
            sumnum = 0
            sum_word = 0
            for t in token:
                for d in word_state_pair:
                    if t == d[0]:
                        sentiment = d[1]
                        if sentiment == "positive":
                            sumnum += 1
                            sum_word += 1
                        elif sentiment == "negative":
                            sumnum += -1
                            sum_word += 1
                        else:
                            sumnum += 0
                            sum_word += 1
                        break
            if sum_word != 0.0:
                sum_more = sumnum / sum_word
                if sum_more >= 0.4:
                    sum_more = 1
                elif (sum_more < 0.4) and (sum_more > -0.5):
                    sum_more = 0
                elif sum_more <= -0.5:
                    sum_more = 0
                else:
                    pass
                    
            varid = k[counter]
            ids.append(varid)
            time_stamp.append(time)
            texts.append(tweet_token)
            pol.append(sum_more)
            counter += 1
            
    data_dict_normal['ids'] = ids
    data_dict_normal['time'] = time_stamp
    data_dict_normal['tweet_text'] = texts
    data_dict_normal['tag'] = pol


length of normal tweets dataset 9056


In [12]:
add_polarity_normal_table()

100%|██████████| 9056/9056 [01:02<00:00, 144.05it/s]


In [13]:
tag_data_depressive = pd.DataFrame.from_dict(data_dict_depressive)

tag_data_depressive.to_csv("2-depressive_tag_dataset.csv",index=False)

In [14]:
tag_data_normal = pd.DataFrame.from_dict(data_dict_normal)

tag_data_normal.to_csv("2-normal_tag_dataset.csv",index=False)

In [15]:
#merge two files
depressive = pd.read_csv("2-depressive_tag_dataset.csv")
normal = pd.read_csv("2-normal_tag_dataset.csv")

In [16]:
depressive.head()

,ids,time,tweet_text,tag
0,1.388080e+18,30-04-2021 10:31,am i mentally ill or is wearing this hair tie ...,0
1,1.388080e+18,30-04-2021 10:31,Bath Salts are used to treat mental and physic...,0
2,1.388080e+18,30-04-2021 10:30,Insomnia and Anxiety really pulled up and beat...,0
3,1.388080e+18,30-04-2021 10:30,hab leider n social anxiety,0
4,1.388080e+18,30-04-2021 10:30,I go to the tip yesterday after my jolly to IK...,0


In [17]:
normal.head()

,ids,time,tweet_text,tag
0,1.390000e+18,30-04-2021 21:41,Yes just fancied it Its blissful,1
1,1.390000e+18,30-04-2021 21:40,Great Relief and Blissful As Bobde out its Gar...,1
2,1.390000e+18,30-04-2021 21:36,My parents did register to stand a better chan...,0
3,1.390000e+18,30-04-2021 21:36,Writing a ship I genuinely love has inverted m...,1
4,1.390000e+18,30-04-2021 21:35,this might help you,1


In [18]:
# df_tweet_tag = df_tweet_tag.dropna(how='any') 
print(np.where(depressive.applymap(lambda x: x == '')))
# print(np.where(pd.isnull(df_tweet_tag)))
#print(np.where(pd.isnull(depressive)))

(array([], dtype=int64), array([], dtype=int64))


In [19]:
final_dataset = depressive.append(normal)

In [20]:
final_dataset.head()

,ids,time,tweet_text,tag
0,1.388080e+18,30-04-2021 10:31,am i mentally ill or is wearing this hair tie ...,0
1,1.388080e+18,30-04-2021 10:31,Bath Salts are used to treat mental and physic...,0
2,1.388080e+18,30-04-2021 10:30,Insomnia and Anxiety really pulled up and beat...,0
3,1.388080e+18,30-04-2021 10:30,hab leider n social anxiety,0
4,1.388080e+18,30-04-2021 10:30,I go to the tip yesterday after my jolly to IK...,0


In [21]:
final_dataset = final_dataset.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [22]:
final_dataset.iloc[10]

ids                                                 1.38808e+18
time                                           30-04-2021 10:30
tweet_text    I have not been present recently I am off work...
tag                                                           0
Name: 10, dtype: object

In [23]:
final_dataset = final_dataset.sample(frac=1)

In [24]:
final_dataset.to_csv('final_dataset.csv', index=False, encoding='utf-8')

In [25]:
#checking
print("depressive file length :",len(depressive))
print("normal file length :",len(normal))
print("final dataset file length :",len(depressive)+len(normal))
print("final dataset after combined length :" ,len(final_dataset))

depressive file length : 7465
normal file length : 9056
final dataset file length : 16521
final dataset after combined length : 16521
